In [22]:
from pyspark.sql import SparkSession
from pyiceberg.catalog import load_catalog
from dotenv import load_dotenv
import os
import ollama
import torch


load_dotenv()

# Get variables from environment
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
jdbc_jar = os.getenv('JDBC_JAR_PATH')
os.makedirs("/tmp/iceberg-warehouse", exist_ok=True)


spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.jars", jdbc_jar) \
    .getOrCreate()



spark_db_conf = {
    "url":f"jdbc:postgresql://{db_host}:{db_port}/{db_name}",
    "user":db_user,
    "pw": db_password
    
}

print(spark_db_conf['url'])

pgsql_details = spark.read \
    .format("jdbc") \
    .option("url", spark_db_conf['url']) \
    .option("query", "SELECT version()") \
    .option("user", spark_db_conf['user']) \
    .option("password", spark_db_conf['pw']) \
    .option("driver", "org.postgresql.Driver") \
    .load()
    

catalog = load_catalog(
    "postgres",
    **{
        "type": "sql",
        "uri": f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}",
        "warehouse": "/tmp/iceberg-warehouse"
    }
)
    
pgsql_details.show()

response = ollama.chat(model='gpt-oss:latest', messages=[
    {
        'role': 'user',
        'content': 'Are you working?',
    },
], keep_alive=0)


print(response)

jdbc:postgresql://172.22.0.9:5432/spark_db
+--------------------+
|             version|
+--------------------+
|PostgreSQL 15.14 ...|
+--------------------+

model='gpt-oss:latest' created_at='2026-02-01T17:12:37.105412813Z' done=True done_reason='stop' total_duration=5174728681 load_duration=3074788600 prompt_eval_count=71 prompt_eval_duration=509469026 eval_count=107 eval_duration=1589893404 message=Message(role='assistant', content='Yes—I’m here and ready to help! What do you need?', thinking='The user asks: "Are you working?" That is ambiguous. Could be a joke about AI. Might want to respond: "Yes, I\'m here, how can I help?" The user might be asking whether I am currently functioning or "working" in the sense of being employed. So answer politely. It\'s an informal question. Should be short and helpful. Probably mention I\'m an AI and always ready to assist.', images=None, tool_name=None, tool_calls=None) logprobs=None


Help on method chat in module ollama._client:

chat(model: str = '', messages: Optional[Sequence[Union[Mapping[str, Any], ollama._types.Message]]] = None, *, tools: Optional[Sequence[Union[Mapping[str, Any], ollama._types.Tool, Callable]]] = None, stream: bool = False, think: Union[bool, Literal['low', 'medium', 'high'], NoneType] = None, logprobs: Optional[bool] = None, top_logprobs: Optional[int] = None, format: Union[Literal['', 'json'], dict[str, Any], NoneType] = None, options: Union[Mapping[str, Any], ollama._types.Options, NoneType] = None, keep_alive: Union[float, str, NoneType] = None) -> Union[ollama._types.ChatResponse, collections.abc.Iterator[ollama._types.ChatResponse]] method of ollama._client.Client instance
    Create a chat response using the requested model.
    
    Args:
      tools:
        A JSON schema as a dict, an Ollama Tool or a Python Function.
        Python functions need to follow Google style docstrings to be converted to an Ollama Tool.
        For mor